In [1]:
import sys
import glob
import os
import shutil

from tqdm import tqdm
from osgeo import gdal

In [2]:
wbt_path = "C:/Users/holgerv/WBT"
if wbt_path not in sys.path:
    sys.path.insert(1, wbt_path)

In [3]:
from whitebox_tools import WhiteboxTools

In [4]:
whitebox = WhiteboxTools()
whitebox.set_whitebox_dir(wbt_path)

In [5]:
whitebox.set_verbose_mode(False)

0

In [6]:
whitebox.set_compress_rasters(True)

0

In [7]:
basepath = "D:/users/holgerv/Ditches"

# Prepare DEM files for WBT

WBT does not work with compressed floating point raster files that have been written with a predictor. Therefore, we have to use GDAL to translate the existing DEM files into versions without the predictor.

In [8]:
# DEM files
dem_files = glob.glob(f"{basepath}/original/DEM/dem_1m_20231203/*_dem_1m.tif")

In [9]:
# Output directory
out_dirpath = f"{basepath}/working/deep_learning/data/dem_1m_wbt"
if os.path.exists(out_dirpath):
    shutil.rmtree(out_dirpath)
os.mkdir(out_dirpath)

In [10]:
%%time

# Write DEM files without predictor
for dem_file in tqdm(dem_files, position=0, leave=True):
    out_basename = "_".join([os.path.basename(dem_file).split(".")[0], "wbt"]) + ".tif"
    out_filename = os.path.join(out_dirpath, out_basename)
    ds = gdal.Translate(out_filename, dem_file, options=gdal.TranslateOptions(outputType=gdal.GDT_Float32, creationOptions=["compress=lzw"]))
    ds = None

  0%|                                                                                                                                                                                                              | 0/2074 [00:00<?, ?it/s]C:\Users\holgerv\micromamba\envs\tf\lib\site-packages\osgeo\gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2074/2074 [45:59<00:00,  1.33s/it]

CPU times: total: 35min 30s
Wall time: 45min 59s


# Apply HPMF to DEM files

In [11]:
# High-pass median filter from WBT
def high_pass_median_filter(in_filename: str, out_filename: str, filterx: int, filtery: int, sig_digits=2):
    tool_name = "HighPassMedianFilter"
    args = [f"--input={in_filename}", f"--output={out_filename}", f"--filterx={filterx}", f"--filtery={filtery}", f"--sig_digits={sig_digits}"]
    whitebox.run_tool(tool_name, args)
    return

In [12]:
# Generate HPMF raster from DEM
def generate_hpmf_raster(fp_dem: str, out_dirpath: str, filterx: int, filtery: int, sig_digits=2):
    out_basename = "_".join([os.path.basename(fp_dem).split(".")[0], "hpmf"]) + ".tif"
    fp_hpmf = os.path.join(out_dirpath, out_basename)
    high_pass_median_filter(fp_dem, fp_hpmf, filterx, filtery, sig_digits)
    return

In [13]:
# DEM files
dem_files = glob.glob(f"{basepath}/working/deep_learning/data/dem_1m_wbt/*.tif")

In [14]:
# Output directory
out_dirpath = f"{basepath}/working/deep_learning/data/dem_1m_hpmf"
if os.path.exists(out_dirpath):
    shutil.rmtree(out_dirpath)
os.mkdir(out_dirpath)

In [15]:
# Filter dimensions
filterx = 5
filtery = 5

In [16]:
# Generate HPMF rasters
for dem_file in tqdm(dem_files, total=len(dem_files), position=0, leave=True):
    generate_hpmf_raster(dem_file, out_dirpath, filterx, filtery)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2073/2073 [3:21:30<00:00,  5.83s/it]
